<a href="https://colab.research.google.com/github/SharuGitHubSpace/SentimentAnalysis/blob/main/SentimentAnalysis_Trial1_Review_Tuesday_Saranya_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import requests
from tqdm import tqdm
from random import sample

In [ ]:
#Start by connecting gdrive into the google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api_key = ''
with open('/content/drive/MyDrive/Dataset/api_key.txt') as f:
    api_key = f.readlines()
key = api_key[0]

In [ ]:
api_key = ''
with open('/content/drive/MyDrive/Dataset/api_url.txt') as f:
    api_url = f.readlines()

url = "http://" + "148.113.143.41" + ":49152/tLM"
url1 = "http://" + "148.113.138.69" + ":49152/tLM"
url_2 = "http://" + "15.235.41.234" + ":49152/tLM"

(X, Y), (dev_X, dev_Y), labels = pickle.load(open("/content/drive/MyDrive/Dataset/SA-GENERAL-Dataset.pkl", "rb"))

In [ ]:
url

'http://148.113.143.41:49152/tLM'

In [ ]:
batch_size = 1000

_r = sample(list(range(len(X))), k = batch_size * 50)
X = [X[i] for i in _r]
Y = [Y[i] for i in _r]

_r = sample(list(range(len(dev_X))), k = batch_size * 50)
dev_X = [dev_X[i] for i in _r]
dev_Y = [dev_Y[i] for i in _r]

In [ ]:
correct = 0
total = 0

y_predL = []
y_predL_review = []
y_predL_emotion = []

y_trueL = []

y_prob = []
y_prob_review = []
y_prob_emotion = []

y_NeutralProb_reviewL = []
for i in tqdm(range(0, len(dev_X), batch_size)):

    output_obj = requests.post(url, json={'API-KEY': key, 'X': dev_X[i:i + batch_size]}).json()
    output_obj2 = requests.post(url1, json={'API-KEY': key, 'X': dev_X[i:i + batch_size]}).json()
    output_obj3 = requests.post(url_2, json={'API-KEY': key, 'X': dev_X[i:i + batch_size]}).json()

    y_pred = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[1] for o in output_obj))
    y_pred_review = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[1] for o in output_obj2))
    y_pred_emotion = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[1] for o in output_obj3))


    y_pred_prob = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[0] for o in output_obj))
    y_pred_prob_review = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[0] for o in output_obj2))
    y_pred_prob_emotion = ((max(((v, k) for k, v in o['extracted'][-2][1].items()))[0] for o in output_obj3))


    for j in output_obj2:
      #print(j['extracted'][-2][1]['[Sentiment=Neutral]'])
      y_NeutralProb_reviewL.append(j['extracted'][-2][1]['[Sentiment=Neutral]'])


    y_true = ((t[0] for t in dev_Y[i:i + batch_size]))

    for a in zip(y_true):
      y_trueL.append(a[0])

    for b in zip(y_pred):
      y_predL.append(b[0])

    for c in zip(y_pred_prob):
      y_prob.append(c[0])

    for a in zip(y_pred_review):
      y_predL_review.append(a[0])

    for a in zip(y_pred_emotion):
      y_predL_emotion.append(a[0])

    for c in zip(y_pred_prob_review):
      y_prob_review.append(c[0])

    for c in zip(y_pred_prob_emotion):
      y_prob_emotion.append(c[0])

100%|██████████| 50/50 [28:58<00:00, 34.77s/it]


In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_trueL, y_predL))

from sklearn.metrics import classification_report

print(classification_report(y_trueL, y_predL))

0.86174


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

    [Sentiment=Negative]       0.83      0.92      0.87     19199
     [Sentiment=Neutral]       0.00      0.00      0.00      3840
    [Sentiment=Positive]       0.89      0.94      0.91     26954
[Sentiment=VeryNegative]       0.00      0.00      0.00         4
[Sentiment=VeryPositive]       0.00      0.00      0.00         3

                accuracy                           0.86     50000
               macro avg       0.34      0.37      0.36     50000
            weighted avg       0.80      0.86      0.83     50000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import copy
exp_list = copy.copy(y_predL)

In [ ]:
import pandas as pd

sentiment = pd.DataFrame(
    {'text': dev_X,
     'gen_label': y_predL,
     'revw_label': y_predL_review,
     'emot_label': y_predL_emotion,
     'true': y_trueL,
     'gen_prob': y_prob,
     'revw_prob': y_prob_review,
     'revw_prod_Neutral' : y_NeutralProb_reviewL,
     'emot_prob': y_prob_emotion,
     'exp' : y_predL
    })

#sentiment.to_csv('/content/drive/MyDrive/sentimentWed.csv')

In [ ]:
sentiment

,text,gen_label,revw_label,emot_label,true,gen_prob,revw_prob,revw_prod_Neutral,emot_prob,exp
0,[an anti-vibration device according to the pre...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Anger],[Sentiment=Neutral],0.992286,0.942991,0.056179,0.862903,[Sentiment=Positive]
1,[No Positive],[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Neutral],[Sentiment=Negative],0.987298,0.992756,0.004417,0.461283,[Sentiment=Negative]
2,[Out of the top 24 lesbian films in my library...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.988805,0.735315,0.000951,0.999703,[Sentiment=Positive]
3,[Our booking confirmation from Booking com sho...,[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Joy],[Sentiment=Negative],0.984476,0.959813,0.004543,0.964457,[Sentiment=Negative]
4,[breakfast was plentiful and of good quality s...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992877,0.991968,0.005855,0.998638,[Sentiment=Positive]
...,...,...,...,...,...,...,...,...,...,...
49995,[He points out that the research is free of ch...,[Sentiment=Negative],[Sentiment=Neutral],[Sentiment=Joy],[Sentiment=Neutral],0.879538,0.594413,0.594413,0.900025,[Sentiment=Negative]
49996,[no negative],[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992555,0.995435,0.002911,0.495467,[Sentiment=Positive]
49997,[It's a much more emotional journey than what ...,[Sentiment=Positive],[Sentiment=VeryPositive],[Sentiment=Fear],[Sentiment=Positive],0.991251,0.609843,0.002214,0.955996,[Sentiment=Positive]
49998,[excellent location friendly staff],[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992445,0.992750,0.002713,0.999363,[Sentiment=Positive]


In [ ]:
for index, row in sentiment.iterrows():

    if(row['gen_label']) != (row['revw_label']):
      if(row['revw_label'] =='[Sentiment=Neutral]'):
        exp_list[index] = '[Sentiment=Neutral]'

In [ ]:
for index, row in sentiment.iterrows():

    if(row['gen_label']) != (row['emot_label']):
      if(row['emot_label'] =='[Sentiment=Neutral]'):
        exp_list[index] = '[Sentiment=Neutral]'

In [ ]:
for index, row in sentiment.iterrows():
    if(row['revw_prod_Neutral']) > 0.30 :
        exp_list[index] = '[Sentiment=Neutral]'

In [ ]:
pos_mapping_review = {
    '[Sentiment=VeryNegative]': '[Sentiment=Negative]',
    '[Sentiment=VeryPositive]': '[Sentiment=Positive]'
}

new_true = [pos_mapping_review.get(item, item) for item in y_trueL]
new_pred = [pos_mapping_review.get(item, item) for item in exp_list]

[Emotion Classification](https://en.wikipedia.org/wiki/Emotion_classification#/media/File:Circumplex_model_of_emotion.svg)

Strategy 2
- Convert all 29 Emotions to 3 - classes ( postivie ,Negative and Neutral)
- Convert 5 reviews into 3 - classes ( postivie ,Negative and Neutral)




In [ ]:
import copy
exp_list_derived = copy.copy(y_predL)
exp_list_RevwDerived = copy.copy(y_predL_review)
exp_list_EmotDerived = copy.copy(y_predL_emotion)

In [ ]:
pos_mapping_emotion = {
     '[Sentiment=Admiration]':  '[Sentiment=Positive]',
     '[Sentiment=Amusement]':  '[Sentiment=Positive]',
     '[Sentiment=Anger]': '[Sentiment=Negative]',
     '[Sentiment=Annoyance]': '[Sentiment=Negative]' ,
     '[Sentiment=Anticipation]':  '[Sentiment=Neutral]' ,
     '[Sentiment=Approval]': '[Sentiment=Neutral]' ,
     '[Sentiment=Caring]': '[Sentiment=Positive]' ,
     '[Sentiment=Confusion]':  '[Sentiment=Neutral]',
     '[Sentiment=Curiosity]': '[Sentiment=Neutral]' ,
     '[Sentiment=Desire]': '[Sentiment=Neutral]' ,
     '[Sentiment=Disappointment]': '[Sentiment=Negative]' ,
     '[Sentiment=Disapproval]': '[Sentiment=Negative]' ,
     '[Sentiment=Disgust]': '[Sentiment=Negative]',
     '[Sentiment=Embarrassment]': '[Sentiment=Negative]',
     '[Sentiment=Excitement]': '[Sentiment=Positive]',
     '[Sentiment=Fear]': '[Sentiment=Negative]',
     '[Sentiment=Gratitude]': '[Sentiment=Positive]' ,
     '[Sentiment=Grief]': '[Sentiment=Negative]',
     '[Sentiment=Joy]':  '[Sentiment=Positive]',
     '[Sentiment=Love]': '[Sentiment=Positive]' ,
     '[Sentiment=Nervousness]':  '[Sentiment=Negative]',
     '[Sentiment=Optimism]': '[Sentiment=Positive]' ,
     '[Sentiment=Pride]': '[Sentiment=Negative]',
     '[Sentiment=Realization]': '[Sentiment=Neutral]' ,
     '[Sentiment=Relief]': '[Sentiment=Neutral]' ,
     '[Sentiment=Remorse]': '[Sentiment=Negative]',
     '[Sentiment=Sadness]': '[Sentiment=Negative]',
     '[Sentiment=Surprise]': '[Sentiment=Positive]',
     '[Sentiment=Trust]':  '[Sentiment=Positive]',
     '[Sentiment=Neutral]' : '[Sentiment=Neutral]'

}

new_true_emotionDerived = [pos_mapping_emotion.get(item, item) for item in y_trueL]
new_pred_emotionDerived = [pos_mapping_emotion.get(item, item) for item in exp_list_EmotDerived]


NameError: ignored

In [ ]:
pos_mapping_review = {
    '[Sentiment=VeryNegative]': '[Sentiment=Negative]',
    '[Sentiment=VeryPositive]': '[Sentiment=Positive]'
}

new_true_reviewDerived = [pos_mapping_review.get(item, item) for item in y_trueL]
new_pred_reviewDerived = [pos_mapping_review.get(item, item) for item in exp_list_RevwDerived]

In [ ]:
pos_mapping_review = {
    '[Sentiment=VeryNegative]': '[Sentiment=Negative]',
    '[Sentiment=VeryPositive]': '[Sentiment=Positive]'
}

test = [pos_mapping_review.get(item, item) for item in ['[Sentiment=VeryPositive]']]
test

['[Sentiment=Positive]']

In [ ]:
sentiment['revw_label_dervd'] = new_pred_reviewDerived
sentiment['emot_label_dervd'] = new_pred_emotionDerived

In [ ]:
sentiment

,text,gen_label,revw_label,emot_label,true,gen_prob,revw_prob,revw_prod_Neutral,emot_prob,exp,revw_label_dervd,emot_label_dervd
0,[an anti-vibration device according to the pre...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Anger],[Sentiment=Neutral],0.992286,0.942991,0.056179,0.862903,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Negative]
1,[No Positive],[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Neutral],[Sentiment=Negative],0.987298,0.992756,0.004417,0.461283,[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Neutral]
2,[Out of the top 24 lesbian films in my library...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.988805,0.735315,0.000951,0.999703,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Positive]
3,[Our booking confirmation from Booking com sho...,[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Joy],[Sentiment=Negative],0.984476,0.959813,0.004543,0.964457,[Sentiment=Negative],[Sentiment=Negative],[Sentiment=Positive]
4,[breakfast was plentiful and of good quality s...,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992877,0.991968,0.005855,0.998638,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Positive]
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,[He points out that the research is free of ch...,[Sentiment=Negative],[Sentiment=Neutral],[Sentiment=Joy],[Sentiment=Neutral],0.879538,0.594413,0.594413,0.900025,[Sentiment=Negative],[Sentiment=Neutral],[Sentiment=Positive]
49996,[no negative],[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992555,0.995435,0.002911,0.495467,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Positive]
49997,[It's a much more emotional journey than what ...,[Sentiment=Positive],[Sentiment=VeryPositive],[Sentiment=Fear],[Sentiment=Positive],0.991251,0.609843,0.002214,0.955996,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Negative]
49998,[excellent location friendly staff],[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Joy],[Sentiment=Positive],0.992445,0.992750,0.002713,0.999363,[Sentiment=Positive],[Sentiment=Positive],[Sentiment=Positive]


In [ ]:
exp_list_derived

['[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Negative]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',
 '[Sentiment=Negative]',
 '[Sentiment=Negative]',
 '[Sentiment=Negative]',
 '[Sentiment=Positive]',


In [ ]:
# voting for Labels - Neutral

In [ ]:
import numpy as np
rep = {'[Sentiment=Positive]':1,'[Sentiment=Negative]':-1,'[Sentiment=Neutral]':0}

for index, row in sentiment.iterrows():
    #if ((((row['gen_label']) != (row['revw_label_dervd'])) and (row['revw_label_dervd'] == '[Sentiment=Neutral]') and (row['revw_prob'] > 0.70))):
    if ((((row['gen_label']) != (row['revw_label_dervd'])) and (row['revw_label_dervd'] == '[Sentiment=Neutral]'))):
        #print(row['revw_label_dervd'])
        #print(  (row['revw_prob'] > 0.90))
        exp_list_derived[index] = '[Sentiment=Neutral]'
    #elif( (((row['gen_label']) != (row['emot_label_dervd'])) and (row['emot_label_dervd'] == '[Sentiment=Neutral]') and (row['emot_prob'] > 0.70))):
    elif( (((row['gen_label']) != (row['emot_label_dervd'])) and (row['emot_label_dervd'] == '[Sentiment=Neutral]') )):
        print(row['emot_prob'] > 0.90)
        #print(row['emot_label_dervd'])
        exp_list_derived[index] = '[Sentiment=Neutral]'



Streaming output truncated to the last 5000 lines.
False
False
True
True
False
False
True
True
False
True
True
True
True
False
False
True
False
True
True
False
False
False
False
True
True
False
False
True
False
False
True
True
True
False
False
False
True
True
False
True
False
False
True
True
False
False
True
True
True
True
False
False
False
True
True
True
True
False
False
True
False
True
False
True
True
False
True
False
True
True
True
True
True
True
True
True
True
False
True
False
False
True
True
False
True
False
False
False
False
True
False
False
False
True
True
True
True
True
False
True
False
True
True
False
False
False
True
True
True
False
False
False
True
True
True
False
True
True
False
True
True
False
False
False
False
True
False
True
True
False
False
False
True
False
False
True
True
True
False
True
True
False
False
False
False
True
False
False
False
True
True
False
False
False
True
False
False
True
False
True
True
True
True
False
False
True
True
True
False
True
True
False
True
Fa

In [ ]:
for index, row in sentiment.iterrows():
    if(row['revw_prod_Neutral']) > 0.30:
        exp_list_derived[index] = '[Sentiment=Neutral]'

In [ ]:
sentiment['label_voted'] = exp_list_derived

In [ ]:
sentiment
location = '/content/drive/MyDrive/Dataset/'

with pd.ExcelWriter(location+'output_AnalysisSA.xlsx') as excel_writer:
    sentiment.to_excel(excel_writer, sheet_name='Review - Sentiment Analysis', index=False)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_trueL, y_predL))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

    [Sentiment=Negative]       0.83      0.92      0.87     19199
     [Sentiment=Neutral]       0.00      0.00      0.00      3840
    [Sentiment=Positive]       0.89      0.94      0.91     26954
[Sentiment=VeryNegative]       0.00      0.00      0.00         4
[Sentiment=VeryPositive]       0.00      0.00      0.00         3

                accuracy                           0.86     50000
               macro avg       0.34      0.37      0.36     50000
            weighted avg       0.80      0.86      0.83     50000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_trueL, exp_list_derived))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          precision    recall  f1-score   support

    [Sentiment=Negative]       0.87      0.63      0.73     19199
     [Sentiment=Neutral]       0.16      0.46      0.23      3840
    [Sentiment=Positive]       0.92      0.85      0.88     26954
[Sentiment=VeryNegative]       0.00      0.00      0.00         4
[Sentiment=VeryPositive]       0.00      0.00      0.00         3

                accuracy                           0.73     50000
               macro avg       0.39      0.39      0.37     50000
            weighted avg       0.84      0.73      0.77     50000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_trueL, y_predL_emotion))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

                          precision    recall  f1-score   support

  [Sentiment=Admiration]       0.00      0.00      0.00         0
       [Sentiment=Anger]       0.00      0.00      0.00         0
        [Sentiment=Fear]       0.00      0.00      0.00         0
         [Sentiment=Joy]       0.00      0.00      0.00         0
        [Sentiment=Love]       0.00      0.00      0.00         0
    [Sentiment=Negative]       0.00      0.00      0.00     19199
     [Sentiment=Neutral]       0.11      0.17      0.13      3840
    [Sentiment=Positive]       0.00      0.00      0.00     26954
     [Sentiment=Sadness]       0.00      0.00      0.00         0
    [Sentiment=Surprise]       0.00      0.00      0.00         0
[Sentiment=VeryNegative]       0.00      0.00      0.00         4
[Sentiment=VeryPositive]       0.00      0.00      0.00         3

                accuracy                           0.01     50000
               macro avg       0.01      0.01      0.01     50000
        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_trueL, y_predL_review))

                          precision    recall  f1-score   support

    [Sentiment=Negative]       0.90      0.78      0.84     19199
     [Sentiment=Neutral]       0.24      0.25      0.24      3840
    [Sentiment=Positive]       0.90      0.87      0.89     26954
[Sentiment=VeryNegative]       0.00      0.00      0.00         4
[Sentiment=VeryPositive]       0.00      0.00      0.00         3

                accuracy                           0.79     50000
               macro avg       0.41      0.38      0.39     50000
            weighted avg       0.85      0.79      0.82     50000

